In [18]:
import random
import copy

In [19]:
class Individual:
    def __init__(self, genetic_code, graph, fitness=None):
        self.genetic_code = genetic_code
        self.graph = graph
        self.nodes = list(self.graph.keys())
        self.num_nodes = len(self.graph)
        self.num_dominated = None
        self.total_selected = None

        if fitness is None:
            self.fitness = self.calculate_fitness()
        else:
            self.fitness = fitness

    def __str__(self):
        return f"Gen: {''.join(map(str, self.genetic_code))} | Fitness: {self.fitness} | Selected: {sum(self.genetic_code)}"

    def __repr__(self):
        return self.__str__()

    def calculate_fitness(self):
        dominated = set()

        for i, gene in enumerate(self.genetic_code):
            if gene == 1:
                node = self.nodes[i]
                dominated.add(node)
                for neighbor in self.graph[node]:
                    dominated.add(neighbor)
        num_dominated = len(dominated)
        total_selected = sum(self.genetic_code)

        if num_dominated == self.num_nodes:
            fitness = self.num_nodes + (self.num_nodes - total_selected)
        else:
            fitness = num_dominated

        self.total_selected = total_selected
        self.num_dominated = num_dominated

        return fitness

In [20]:
def initial_population(graph, population_size):
    population = []
    n = len(graph)
    for _ in range(population_size):
        genetic_code = [random.randint(0, 1) for _ in range(n)]
        individual = Individual(genetic_code, graph)
        population.append(individual)
    return population

In [21]:
def roulette_selection(population):
    total_fitness = sum(individual.fitness for individual in population)
    if total_fitness == 0:
        return random.choice(population)
    winner = random.choices(population, weights=[individual.fitness for individual in population], k=1)[0]
    return winner

In [22]:
def tournament_selection(population, tournament_size):
    selected = random.sample(population, tournament_size)
    winner = max(selected, key=lambda x: (x.fitness, -x.total_selected))
    return winner

In [23]:
def crossover(parent1, parent2):
    n = parent1.num_nodes

    break_point = random.randint(1, n - 1)
    child1_code = parent1.genetic_code[:break_point] + parent2.genetic_code[break_point:]
    child2_code = parent2.genetic_code[:break_point] + parent1.genetic_code[break_point:]

    child1 = Individual(child1_code, parent1.graph)
    child2 = Individual(child2_code, parent2.graph)

    return child1, child2

In [24]:
def mutate(individual, mutation_rate):
    if random.random() < mutation_rate:
        index = random.randrange(individual.num_nodes)
        individual.genetic_code[index] = 1 - individual.genetic_code[index]
        individual.fitness = individual.calculate_fitness()
    return individual

In [25]:
def ga(graph, population_size, mutation_rate, tournament_size, max_iterations, selection_method, elitism_size):
    if selection_method not in ['roulette_selection', 'tournament_selection']:
        raise ValueError("Invalid selection method")

    population = initial_population(graph, population_size)

    for generation in range(1, max_iterations + 1):
        population.sort(key=lambda x: (-x.fitness, x.total_selected))
        best_individual = population[0]
        print(f"Generation {generation}: {best_individual}")

        new_population = []
        elites = [copy.deepcopy(individual) for individual in population[:elitism_size]]
        new_population.extend(elites)

        while len(new_population) < population_size:
            if selection_method == 'roulette_selection':
                parent1 = roulette_selection(population)
                parent2 = roulette_selection(population)
            else:
                parent1 = tournament_selection(population, tournament_size)
                parent2 = tournament_selection(population, tournament_size)

            child1, child2 = crossover(parent1, parent2)
            child1 = mutate(child1, mutation_rate)
            child2 = mutate(child2, mutation_rate)

            children = [child1, child2]
            children.sort(key=lambda x: (-x.fitness, x.total_selected))

            for child in children:
                if len(new_population) < population_size:
                    new_population.append(child)
                else:
                    break
        population = new_population

    dominating_individuals = [individual for individual in population if
                              individual.num_dominated == individual.num_nodes]

    if dominating_individuals:
        best = max(dominating_individuals, key=lambda x: (x.fitness, -x.total_selected))
    else:
        best = max(population, key=lambda x: (x.fitness, -x.total_selected))

    print(f"Best individual after {max_iterations} iterations: {best}")

    return best.genetic_code, best.fitness

In [26]:
if __name__ == '__main__':
    g = {
        1: [2, 5],
        2: [1, 3, 5],
        3: [2, 4, 6],
        4: [3],
        5: [1, 2, 6],
        6: [3, 5]
    }

    best_code, best_fitness = ga(
        graph=g,
        population_size=50,
        mutation_rate=0.1,
        tournament_size=3,
        max_iterations=100,
        selection_method='tournament_selection',
        elitism_size=2
    )

    nodes = sorted(g.keys())
    print("Best genetic code (dominating set):", best_code)

    selected_vertices = [nodes[i] for i, bit in enumerate(best_code) if bit == 1]
    print("Selected vertices:", selected_vertices)
    print("Fitness:", best_fitness)

Generation 1: Gen: 011000 | Fitness: 10 | Selected: 2
Generation 2: Gen: 011000 | Fitness: 10 | Selected: 2
Generation 3: Gen: 011000 | Fitness: 10 | Selected: 2
Generation 4: Gen: 011000 | Fitness: 10 | Selected: 2
Generation 5: Gen: 011000 | Fitness: 10 | Selected: 2
Generation 6: Gen: 011000 | Fitness: 10 | Selected: 2
Generation 7: Gen: 011000 | Fitness: 10 | Selected: 2
Generation 8: Gen: 011000 | Fitness: 10 | Selected: 2
Generation 9: Gen: 011000 | Fitness: 10 | Selected: 2
Generation 10: Gen: 011000 | Fitness: 10 | Selected: 2
Generation 11: Gen: 011000 | Fitness: 10 | Selected: 2
Generation 12: Gen: 011000 | Fitness: 10 | Selected: 2
Generation 13: Gen: 011000 | Fitness: 10 | Selected: 2
Generation 14: Gen: 011000 | Fitness: 10 | Selected: 2
Generation 15: Gen: 011000 | Fitness: 10 | Selected: 2
Generation 16: Gen: 011000 | Fitness: 10 | Selected: 2
Generation 17: Gen: 011000 | Fitness: 10 | Selected: 2
Generation 18: Gen: 011000 | Fitness: 10 | Selected: 2
Generation 19: Gen:

In [27]:
import networkx as nx

In [28]:
medium_graph = {
    1: [2, 27],
    2: [1, 30, 7],
    3: [30, 24, 25],
    4: [5, 19],
    5: [4, 8],
    6: [31, 15, 16],
    7: [2, 29, 17, 32],
    8: [5, 9, 17, 19],
    9: [8, 10],
    10: [9, 17],
    11: [12, 13, 24],
    12: [11, 14, 26],
    13: [11, 23, 28],
    14: [12, 29, 18],
    15: [6, 20, 21, 16],
    16: [6, 15, 31, 32],
    17: [7, 8, 10, 27],
    18: [14, 29, 19],
    19: [4, 8, 18],
    20: [15, 21],
    21: [15, 20, 22],
    22: [21, 29, 26],
    23: [13, 28, 24],
    24: [3, 11, 23, 25],
    25: [3, 24],
    26: [12, 22, 29],
    27: [1, 17],
    28: [13, 23],
    29: [7, 14, 18, 22, 26, 32],
    30: [2, 3, 31, 32],
    31: [6, 16, 30],
    32: [7, 16, 29, 30]
}

In [36]:
best_code, best_fitness = ga(
    graph=medium_graph,
    population_size=5000,
    mutation_rate=0.2,
    tournament_size=100,
    max_iterations=20000,
    selection_method='tournament_selection',
    elitism_size=2500
)

nodes = sorted(medium_graph.keys())
print("Best genetic code (dominating set):", best_code)

selected_vertices = [nodes[i] for i, bit in enumerate(best_code) if bit == 1]
print("Selected vertices:", selected_vertices)
print("Fitness:", best_fitness)

Generation 1: Gen: 00011000110011100000001100101100 | Fitness: 52 | Selected: 12
Generation 2: Gen: 01010000100000011010100101011000 | Fitness: 53 | Selected: 11
Generation 3: Gen: 10010000111000000001001010001010 | Fitness: 54 | Selected: 10
Generation 4: Gen: 00010000100100100000001100101100 | Fitness: 55 | Selected: 9
Generation 5: Gen: 00010000100100100000001100101100 | Fitness: 55 | Selected: 9
Generation 6: Gen: 00010000100100100000001100101100 | Fitness: 55 | Selected: 9
Generation 7: Gen: 00010000100100100000001100101100 | Fitness: 55 | Selected: 9
Generation 8: Gen: 00010000100100100000001100101100 | Fitness: 55 | Selected: 9
Generation 9: Gen: 00010000100100100000001100101100 | Fitness: 55 | Selected: 9
Generation 10: Gen: 00010000100100100000001100101100 | Fitness: 55 | Selected: 9
Generation 11: Gen: 00010000100100100000001100101100 | Fitness: 55 | Selected: 9
Generation 12: Gen: 00010000100100100000001100101100 | Fitness: 55 | Selected: 9
Generation 13: Gen: 00010000100100

KeyboardInterrupt: 

In [37]:
best_code = [int(i) for i in "00010000100100100000001100101100"]
nodes = sorted(medium_graph.keys())
selected_vertices = [nodes[i] for i, bit in enumerate(best_code) if bit == 1]
print("Selected vertices:", selected_vertices)

Selected vertices: [4, 9, 12, 15, 23, 24, 27, 29, 30]


In [39]:
medium_graph_nx = nx.Graph(medium_graph_dict)
is_dominating = nx.is_dominating_set(medium_graph_nx, selected_vertices)

if is_dominating:
    print(f"{selected_vertices} is dominating set.")

[4, 9, 12, 15, 23, 24, 27, 29, 30] is dominating set.
